In [3]:
from io import StringIO
import sys, getopt
import pandas as pd
import numpy as np
import re
import os
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
from tqdm import tqdm

In [6]:
import glob
text = ""
for i in glob.glob("Data/" + "*.txt"):
    with open(i, 'r') as f:
        text += f.read()

In [7]:
len(text)

5667777

In [8]:
alfabe = 'çığöşüâqwertyuioplkjhgfdsazxcvbnmÇIĞÖŞÜÂQWERTYUIOPLKJHGFDSAZXCVBNM'
alfabe = list(alfabe)
alfabe.append("<start>")
alfabe.append("<end>")
alfabe.append("\n")
alfabe.append("<")
alfabe.append(">")
alfabe.append("\\")
alfabe.append(" ")

In [9]:
chars_set = set(text)

In [10]:
for i in chars_set:
    if i not in alfabe:
        text = text.replace(i, "")

In [11]:
len(text)

5486598

In [12]:
text = text.replace("\n", " \n ")
text = text.replace("<start>", " <start> ")
text = text.replace("<end>", " <end> ")

In [13]:
len(text)

5875624

In [14]:
for k in range(100):
    text = text.replace("  ", " ")
    text = text.replace("   ", " ")
    text = text.replace("    ", " ")

In [15]:
len(text)

5761692

In [16]:
text = text.replace("<start>", "<siir> <start>")
text = text.replace("<end>", "<end> <zero> <zero> <zero> <zero>")

In [17]:
text = text.split("<siir>")

In [18]:
for i in range(len(text)):
    text[i] = text[i].split(" ")

In [19]:
import gensim.models.word2vec as w2v
import multiprocessing
num_features = 300
min_word_count = 1
num_workers = multiprocessing.cpu_count()
context_size = 7
downsampling = 1e-3
seed = 1

w2vmodel = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [20]:
w2vmodel.build_vocab(text)

In [21]:
w2vmodel.train(text, total_examples=w2vmodel.corpus_count, epochs=20)

(15351914, 20134540)

In [22]:
w2vmodel.wv.most_similar('<start>', topn=10)

[('', 0.7769801020622253),
 ('I', 0.4263428747653961),
 ('mevsiminde', 0.35559746623039246),
 ('Günlerden', 0.3504243791103363),
 ('Yorgun', 0.34636354446411133),
 ('şara', 0.34299999475479126),
 ('bulutlarıyla', 0.341490238904953),
 ('Büyükannemi', 0.33779749274253845),
 ('Yeşilçiçeği', 0.33631178736686707),
 ('Nihale', 0.33519837260246277)]

In [23]:
word_to_vec_map = w2vmodel[w2vmodel.wv.vocab]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [24]:
vocab = w2vmodel.wv.vocab

In [25]:
word_to_index={}
index_to_word = {}
for i, w in enumerate(vocab):
    word_to_index[w] = i
    index_to_word[i] = w

In [26]:
word_to_index['bu']

812

In [27]:
index_to_word[2378]

'altında'

In [29]:
import tflearn
from tflearn.data_utils import *
import tensorflow as tf 

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
max_len = 5

In [31]:

g = tflearn.input_data([None, max_len, num_features])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, num_features, activation='linear')
g = tflearn.regression(g, optimizer='adam', loss='mean_square',
                       learning_rate=0.001)


In [32]:
trainX = []
trainY = []
for i in tqdm(text):
    if len(i) > 6:
        for k in range(len(i)-6):
            
            a = []
            for j in range(k,k+5):
                a.append(w2vmodel[i[j]])
            trainX.append(a)
            
            b= w2vmodel[i[k+5]]
            
            trainY.append(b)

  0%|          | 0/7387 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
100%|██████████| 7387/7387 [00:33<00:00, 218.89it/s]


In [ ]:
trainX = np.array(trainX)

In [ ]:
trainY = np.array(trainY)

In [ ]:
trainX.shape, trainY.shape

In [33]:

model = tflearn.DNN(g, tensorboard_verbose=0)

In [ ]:
model.fit(trainX, trainY, validation_set=0.1, n_epoch=51, run_id='siir')

Training Step: 3150  | total loss: 0.00069 | time: 30.916s
| Adam | epoch: 020 | loss: 0.00069 -- iter: 05824/10280


In [36]:
prediction = model.predict(trainX)[0]

In [37]:
w2vmodel.wv.similar_by_vector(prediction)

[('Terketmedi', 0.9986027479171753),
 ('sevdan', 0.9984055161476135),
 ('rüzgarın', 0.9982413053512573),
 ('artık', 0.9981492757797241),
 ('seviyorum', 0.9981076717376709),
 ('ansızın', 0.9979076385498047),
 ('anladığın', 0.9976863861083984),
 ('arıyorum', 0.9976401925086975),
 ('kaldırımları', 0.9976112246513367),
 ('düşmüşüm', 0.9975905418395996)]

In [44]:
w2vmodel.wv.similar_by_vector(trainX[0][0],topn=1)[0][0], w2vmodel.wv.similar_by_vector(trainX[0][1],topn=1)

('', [('<start>', 0.9999999403953552)])

In [45]:
for i in range(5):
    print(w2vmodel.wv.similar_by_vector(trainX[0][i],topn=1)[0][0])


<start>


Ne
hasta
